Baseline_TF
https://www.kaggle.com/code/ivaneleskin/baseline-tf

In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tqdm import tqdm
import json
import gc
from sklearn.model_selection import train_test_split
# import missingno as msno

from pathlib import Path

2023-02-27 12:31:58.882809: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 12:31:59.029560: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-27 12:31:59.504526: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/victor/miniconda3/envs/tf/lib/:/home/victor/miniconda3/envs/tf/lib/:/home/victor/miniconda3/envs/tf/lib/:/home/victor/miniconda3/envs/tf/lib/
2023-02-27 12:31:59.504585: W tensorflow/stream_executor/platfo

In [2]:
DATA_ROOT = Path('..')
DF_TRAIN = DATA_ROOT / 'train.csv'
train = pd.read_csv(DF_TRAIN)

print("train.shape = {} rows, {} cols".format(*train.shape))
train.info()
train.head()


train.shape = 94477 rows, 4 cols
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94477 entries, 0 to 94476
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   path            94477 non-null  object
 1   participant_id  94477 non-null  int64 
 2   sequence_id     94477 non-null  int64 
 3   sign            94477 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.9+ MB


,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [3]:
class CFG:
    data_path = "../"
    quick_experiment = False
    is_training = True
    use_aggregation_dataset = True
    num_classes = 250
    rows_per_frame = 543 

def load_relevant_data_subset_with_imputation(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    data.replace(np.nan, 0, inplace=True)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / CFG.rows_per_frame)
    data = data.values.reshape(n_frames, CFG.rows_per_frame, len(data_columns))
    return data.astype(np.float32)

def read_dict(file_path):
    path = os.path.expanduser(file_path)
    with open(path, "r") as f:
        dic = json.load(f)
    return dic

In [4]:
train = pd.read_csv(f"{CFG.data_path}train.csv")
label_index = read_dict(f"{CFG.data_path}sign_to_prediction_index_map.json")
index_label = {label_index[key]: key for key in label_index}
train["label"] = train["sign"].map(lambda sign: label_index[sign])

In [5]:
if CFG.is_training:
    print('True')
    if CFG.use_aggregation_dataset == True:
        print('True')
        xs, ys = [], []
        num_frames = np.zeros(len(train))
        for i in tqdm(range(len(train))):
            path = f"{CFG.data_path}{train.iloc[i].path}"
            data = load_relevant_data_subset_with_imputation(path)
            ## Mean Aggregation
            xs.append(np.mean(data, axis=0))
            ys.append(train.iloc[i].label)
            num_frames[i] = data.shape[0]
            if CFG.quick_experiment and i == 4999:
                break
        ## Save number of frames of each training sample for data analysis
        train["num_frames"] = num_frames
        X, y = np.array(xs), np.array(ys)
        print(train["num_frames"].describe())
        train.to_csv("train.csv", index=False)
    else:
        X = np.load("../X.npy")
        y = np.load("../y.npy")
    print(X.shape, y.shape)

True
True


100%|████████████████████████████████████████████████████████████████████████| 94477/94477 [04:42<00:00, 334.21it/s]


count    94477.000000
mean        37.935021
std         44.177069
min          2.000000
25%         12.000000
50%         22.000000
75%         44.000000
max        537.000000
Name: num_frames, dtype: float64
(94477, 543, 3) (94477,)


In [6]:
def get_model():
    inputs = tf.keras.Input((543, 3), dtype=tf.float32)
    h1 = tf.keras.layers.Dense(128, activation="relu")(inputs)
    h2 = tf.keras.layers.Dense(64, activation="relu")(h1)
    h3 = tf.keras.layers.Dense(64, activation="relu")(h2)
    
    concat = tf.keras.layers.Concatenate()([inputs, h3])
    vector = tf.keras.layers.Dense(64, activation="relu")(concat)
    vector = tf.keras.layers.Dense(32, activation="relu")(vector)
    vector = tf.keras.layers.Dense(32, activation="relu")(vector)
    vector = tf.keras.layers.Flatten()(vector)
    output = tf.keras.layers.Dense(250, activation="softmax")(vector)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
        metrics=[
            "accuracy",
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top-5-accuracy"),
            tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
        ]
    )
    return model

In [40]:
if CFG.is_training:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
#     del X, y
    gc.collect()
    model = get_model()
    callbacks = [tf.keras.callbacks.ModelCheckpoint("model.h5")]
    model.fit(X_train, y_train, epochs=80, validation_data=(X_val, y_val), batch_size=128, callbacks=callbacks)
else:
    model = tf.keras.models.load_model("../model.h5")
model.summary()

(75581, 543, 3) (75581,) (18896, 543, 3) (18896,)
Epoch 1/80
591/591 [==============================] - 5s 8ms/step - loss: 5.3238 - accuracy: 0.0198 - top-5-accuracy: 0.0754 - sparse_top_k_categorical_accuracy: 0.1264 - val_loss: 5.0131 - val_accuracy: 0.0464 - val_top-5-accuracy: 0.1587 - val_sparse_top_k_categorical_accuracy: 0.2412
Epoch 2/80
591/591 [==============================] - 4s 8ms/step - loss: 4.7703 - accuracy: 0.0736 - top-5-accuracy: 0.2162 - sparse_top_k_categorical_accuracy: 0.3195 - val_loss: 4.6079 - val_accuracy: 0.0946 - val_top-5-accuracy: 0.2659 - val_sparse_top_k_categorical_accuracy: 0.3826
Epoch 3/80
591/591 [==============================] - 4s 8ms/step - loss: 4.3250 - accuracy: 0.1294 - top-5-accuracy: 0.3317 - sparse_top_k_categorical_accuracy: 0.4524 - val_loss: 4.2983 - val_accuracy: 0.1452 - val_top-5-accuracy: 0.3584 - val_sparse_top_k_categorical_accuracy: 0.4786
Epoch 4/80
591/591 [==============================] - 5s 8ms/step - loss: 4.0393 - acc

591/591 [==============================] - 5s 8ms/step - loss: 2.6633 - accuracy: 0.4108 - top-5-accuracy: 0.6886 - sparse_top_k_categorical_accuracy: 0.7808 - val_loss: 3.5357 - val_accuracy: 0.2917 - val_top-5-accuracy: 0.5603 - val_sparse_top_k_categorical_accuracy: 0.6723
Epoch 30/80
591/591 [==============================] - 4s 8ms/step - loss: 2.6441 - accuracy: 0.4147 - top-5-accuracy: 0.6911 - sparse_top_k_categorical_accuracy: 0.7842 - val_loss: 3.5566 - val_accuracy: 0.2740 - val_top-5-accuracy: 0.5469 - val_sparse_top_k_categorical_accuracy: 0.6619
Epoch 31/80
591/591 [==============================] - 5s 8ms/step - loss: 2.6155 - accuracy: 0.4215 - top-5-accuracy: 0.6966 - sparse_top_k_categorical_accuracy: 0.7884 - val_loss: 3.5753 - val_accuracy: 0.2863 - val_top-5-accuracy: 0.5561 - val_sparse_top_k_categorical_accuracy: 0.6659
Epoch 32/80
591/591 [==============================] - 5s 8ms/step - loss: 2.5941 - accuracy: 0.4254 - top-5-accuracy: 0.7007 - sparse_top_k_cate

591/591 [==============================] - 5s 8ms/step - loss: 2.1651 - accuracy: 0.5083 - top-5-accuracy: 0.7741 - sparse_top_k_categorical_accuracy: 0.8485 - val_loss: 3.6096 - val_accuracy: 0.2896 - val_top-5-accuracy: 0.5613 - val_sparse_top_k_categorical_accuracy: 0.6754
Epoch 58/80
591/591 [==============================] - 5s 8ms/step - loss: 2.1480 - accuracy: 0.5099 - top-5-accuracy: 0.7770 - sparse_top_k_categorical_accuracy: 0.8510 - val_loss: 3.9224 - val_accuracy: 0.2940 - val_top-5-accuracy: 0.5612 - val_sparse_top_k_categorical_accuracy: 0.6673
Epoch 59/80
591/591 [==============================] - 5s 8ms/step - loss: 2.1347 - accuracy: 0.5130 - top-5-accuracy: 0.7791 - sparse_top_k_categorical_accuracy: 0.8514 - val_loss: 3.8489 - val_accuracy: 0.2851 - val_top-5-accuracy: 0.5555 - val_sparse_top_k_categorical_accuracy: 0.6653
Epoch 60/80
591/591 [==============================] - 5s 8ms/step - loss: 2.1241 - accuracy: 0.5161 - top-5-accuracy: 0.7801 - sparse_top_k_cate

                                                                                                  
 dense_10 (Dense)               (None, 543, 64)      4352        ['concatenate_1[0][0]']          
                                                                                                  
 dense_11 (Dense)               (None, 543, 32)      2080        ['dense_10[0][0]']               
                                                                                                  
 dense_12 (Dense)               (None, 543, 32)      1056        ['dense_11[0][0]']               
                                                                                                  
 flatten_1 (Flatten)            (None, 17376)        0           ['dense_12[0][0]']               
                                                                                                  
 dense_13 (Dense)               (None, 250)          4344250     ['flatten_1[0][0]']              
          

In [13]:
tf.saved_model.save(model, 'temp/')

INFO:tensorflow:Assets written to: temp/assets


In [39]:
preds = np.argmax(model.predict(X_train),axis=1)
corr = sum(preds == y_train)
corr, corr/len(preds)

2362/2362 [==============================] - 6s 2ms/step


(39688, 0.5251055159365449)

In [16]:
# converter = tf.lite.TFLiteConverter.from_saved_model('temp/') # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)

In [17]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpi1nccvc4/assets


2023-02-27 12:49:56.670859: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-27 12:49:56.670898: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-27 12:49:56.671049: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpi1nccvc4
2023-02-27 12:49:56.673294: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-27 12:49:56.673309: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpi1nccvc4
2023-02-27 12:49:56.680011: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-02-27 12:49:56.743934: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpi1nccvc4
2023-02-27 12:49:56.756675: I tensorflow/cc/saved_model/loader.cc:305] SavedModel load for tags { serve }; Status: success: OK. Took 85627 m

In [44]:
for i in range(1, len(model.layers)): print(i, model.layers[i])

1 <keras.layers.core.dense.Dense object at 0x7f3ae055e550>
2 <keras.layers.core.dense.Dense object at 0x7f3ae055e5e0>
3 <keras.layers.core.dense.Dense object at 0x7f3ae055e940>
4 <keras.layers.merging.concatenate.Concatenate object at 0x7f3ae011c880>
5 <keras.layers.core.dense.Dense object at 0x7f3ae031b250>
6 <keras.layers.core.dense.Dense object at 0x7f3b8044f880>
7 <keras.layers.core.dense.Dense object at 0x7f3b94089ee0>
8 <keras.layers.reshaping.flatten.Flatten object at 0x7f3c0419de50>
9 <keras.layers.core.dense.Dense object at 0x7f3ae00fa070>


In [48]:
def get_inference_model(model):
    inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
    x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
    x = tf.reduce_mean(x, axis=0, keepdims=True)
    
    h1 = model.layers[1](x)
    h2 = model.layers[2](h1)
    h3 = model.layers[3](h2)
    
    concat = model.layers[4]([x, h3])
    vector = model.layers[5](concat)
    vector = model.layers[6](vector)
    vector = model.layers[7](vector)
    vector = model.layers[8](vector)
    x = model.layers[9](vector)
    
    output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
    inference_model = tf.keras.Model(inputs=inputs, outputs=output) 
    inference_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
    return inference_model

In [49]:
# def get_inference_model(model):
#     inputs = tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")
#     x = tf.where(tf.math.is_nan(inputs), tf.zeros_like(inputs), inputs)
#     x = tf.reduce_mean(x, axis=0, keepdims=True)
#     for i in range(1, len(model.layers)):
#         x = model.layers[i](x)
    
#     output = tf.keras.layers.Activation(activation="linear", name="outputs")(x)
#     inference_model = tf.keras.Model(inputs=inputs, outputs=output) 
#     inference_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=["accuracy"])
#     return inference_model

In [50]:
inference_model = get_inference_model(model)
inference_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, 543, 3)]     0           []                               
                                                                                                  
 tf.math.is_nan_2 (TFOpLambda)  (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.zeros_like_2 (TFOpLambda)   (None, 543, 3)       0           ['inputs[0][0]']                 
                                                                                                  
 tf.where_2 (TFOpLambda)        (None, 543, 3)       0           ['tf.math.is_nan_2[0][0]',       
                                                                  'tf.zeros_like_2[0][0]',  

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"
# Save the model.
with open(model_path, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp6z18gb2w/assets


INFO:tensorflow:Assets written to: /tmp/tmp6z18gb2w/assets
2023-02-27 13:12:52.422873: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-27 13:12:52.422909: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-27 13:12:52.423057: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6z18gb2w
2023-02-27 13:12:52.424368: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-27 13:12:52.424384: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp6z18gb2w
2023-02-27 13:12:52.427795: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-02-27 13:12:52.460475: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp6z18gb2w
2023-02-27 13:12:52.467094: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

In [55]:
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path)
found_signatures = list(interpreter.get_signature_list().keys())
prediction_fn = interpreter.get_signature_runner("serving_default")
corr = 0
wrong = []
for i in tqdm(range(len(train))):
    frames = load_relevant_data_subset(f'../{train.iloc[i].path}')
    output = prediction_fn(inputs=frames)
    sign = np.argmax(output["outputs"])
    if index_label[sign] == train.iloc[i].sign:
        corr += 1
    else:
        wrong.append((index_label[sign], train.iloc[i].sign, train.iloc[i].path))

    #     print(f"Predicted label: {index_label[sign]}, Actual Label: {train.iloc[i].sign}")


    total = i

100%|████████████████████████████████████████████████████████████████████████| 94477/94477 [07:32<00:00, 208.95it/s]


In [56]:
r = f'''
{total=}
{corr=}\t Percent: {corr/total:%}
'''
print(r)


total=94476
corr=45869	 Percent: 48.550955%



In [57]:
wrongs = pd.DataFrame(data=wrong, columns=['pred','true','path'])
wrongs.groupby('pred').count().reset_index().sort_values('true')

,pred,true,path
2,airplane,37,37
198,sick,41,41
171,pig,43,43
196,shoe,44,44
232,wait,45,45
...,...,...,...
3,all,563,563
175,pool,663,663
136,look,666,666
1,after,721,721


In [ ]:
X_train.s